# Error in 3D

In this notebook we load the points OpenPTV creates errors in 3D from 
Robin generated 2D and 3D points, see also proPTV_LineOfSight/generate_error.py

and markers_c0.txt, ...


In [22]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

from openptv_python.calibration import Calibration
from openptv_python.parameters import ControlPar, VolumePar, OrientPar

from openptv_python.imgcoord import image_coordinates
from openptv_python.trafo import arr_metric_to_pixel
from openptv_python.orientation import external_calibration, full_calibration


import pathlib, os
working_path = pathlib.Path.cwd()

In [2]:
class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

In [3]:
params = Parameter()

markers = [np.loadtxt('../proPTV_LineOfSight/markers_c'+str(cam)+'.txt') for cam in params.cams]
XYZ = markers[0][:,2:]
xy = markers[0][:,:2]
ID = np.argwhere((XYZ[:,0]>-1))[:,0]

In [4]:
xyz = pd.DataFrame(XYZ, columns=['x','y','z'])
xyz['id'] = ID
# px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()

In [5]:
# First, let's calibrate roughly the cameras

In [6]:

ref_pts = XYZ[[0,721,1409,1462],:]
xyz = pd.DataFrame(ref_pts, columns=['x','y','z'])
xyz['id'] = ID[[0,721,1409,1462]]
# px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()


In [7]:


cal = Calibration().from_file(working_path / "calibration" / "cam{cam_id}.tif.ori", None)
cpar = ControlPar().from_file(working_path / "parameters" / "ptv.par")
vpar = VolumePar().from_file(working_path / "parameters" / "criteria.par")


No addpar file found. Using default values for radial distortion


In [10]:
four_points = xy[[0,721,1409,1462],:]
print(f"{four_points = }")

four_points = array([[ 315., 1996.],
       [2133.,  235.],
       [ 892.,  561.],
       [2215., 2018.]])


In [49]:
external_calibration(cal, ref_pts, four_points, cpar)
print(f"{cal.ext_par = }")

cal.ext_par = rec.array((364.71589352, -7.50430841, 1145.11492808, 0.16157488, 0.23660541, -0.02809712, [[ 0.97175558,  0.02731072,  0.23440396], [ 0.00996682,  0.98764498, -0.15639074], [-0.23577904,  0.15430983,  0.95947732]]),
          dtype=[('x0', '<f8'), ('y0', '<f8'), ('z0', '<f8'), ('omega', '<f8'), ('phi', '<f8'), ('kappa', '<f8'), ('dm', '<f8', (3, 3))])


In [50]:
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.mm),
cpar,
)
four_points - targets

array([[-5.56247218,  4.8153403 ],
       [ 6.53899267, -5.39916791],
       [ 6.43418308,  9.28669656],
       [-7.15152344, -8.41235306]])

In [52]:
def array_to_calibration(x:np.ndarray, cal:Calibration) -> None:
    # cal = Calibration()
    cal.set_pos(x[:3])
    cal.set_angles(x[3:6])
    cal.set_primary_point(x[6:9])
    cal.set_radial_distortion(x[9:12])
    cal.set_decentering(x[12:14])
    cal.set_affine_distortion(x[14:])
    return None

def calibration_to_array(cal:Calibration) -> np.ndarray:
    return np.concatenate([
        cal.get_pos(),
        cal.get_angles(),
        cal.get_primary_point(),
        cal.get_radial_distortion(),
        cal.get_decentering(),
        cal.get_affine(),
    ])

In [53]:
def error_function(x, cal, XYZ, xy, cpar):
    
    array_to_calibration(x, cal)
    
    targets = arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.mm),
    cpar,
    )
    err = np.sum(np.abs(xy - targets))
    # print(err)
    return err

In [58]:

x0 = calibration_to_array(cal)
sol = minimize(error_function, x0, args=(cal, XYZ, xy, cpar), method='Nelder-Mead', tol=1e-4)

23383.523098325884
220632.735304675
22820.82019078411
209476.48441146783
105730.3412080989
134064.53113576982
23128.484216097655
32072.290033757374
26557.432876543753
78555.45004683424
23383.52309561676
23383.522593178954
23386.804895077294
23383.523096934696
23383.52309694563
49016.635068479816
23383.52309832457
246914.28057348554
115646.1818813577
233194.06790725174
109415.85338155612
176443.250736135
67972.66467706094
115589.79409424527
64490.5937753385
103293.69663986462
107121.70080175882
60197.036877715545
103910.95237749019
53412.48660179935
71619.5369469543
36108.60241081372
84376.35438908567
35729.402658283856
78819.38101959071
32770.55621078896
54703.66000913162
33574.031279558374
45328.3557550247
53217.86003732808
31117.71660808354
52352.55132552683
27192.007632986497
36959.54667640296
24291.622360132802
50277.15436624797
22194.28881609947
38116.81363595884
24248.68445953839
47689.01078900685
20871.782287556984
21589.128797236266
31920.007521714302
24515.75953727308
32943.26

In [60]:
array_to_calibration(sol.x, cal)

targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.mm),
cpar,
)
four_points - targets


array([[ 1.4940228 , -3.39689642],
       [-3.20540632,  1.24652253],
       [ 0.70491151,  0.07242609],
       [-1.6004268 ,  0.17023819]])

In [61]:
targets = arr_metric_to_pixel(
    image_coordinates(XYZ, cal, cpar.mm),
cpar,
)


In [62]:

import plotly.figure_factory as ff
# px.scatter(x=xy[:,0], y=xy[:,1], color=ID).show()
fig = ff.create_quiver(x=xy[:,0], y=xy[:,1], u=targets[:,0]-xy[:,0], v=targets[:,1]-xy[:,1], scale=5)
fig.show()


In [70]:
cal.write(working_path / "calibration" / "cam{cam_id}_scipy.ori", working_path / "calibration" / "cam{cam_id}_scipy.addpar")